# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json


# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sorland,67.6670,12.6934,-2.08,42,91,7.93,NO,1678137469
1,1,shelburne,44.0787,-80.2041,0.98,97,100,5.07,CA,1678137679
2,2,ushuaia,-54.8000,-68.3000,11.79,71,40,6.17,AR,1678137539
3,3,vila,42.0304,-8.1588,7.49,77,100,1.36,PT,1678137680
4,4,hermanus,-34.4187,19.2345,18.78,78,36,6.54,ZA,1678137460


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
                    "Lng",
                    "Lat",
                     geo = True,
                     tiles = "OSM",
                     frame_width = 800,
                     frame_height = 600,
                     size = "Humidity",
                     scale =2,
                     color = "City"
                      ) 

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df= city_data_df.loc[(city_data_df["Max Temp"] >= 20) &
                                                 (city_data_df["Max Temp"] < 30) &
                                                 (city_data_df["Humidity"] >= 30 )&
                                                 (city_data_df["Humidity"] < 60 ) &
                                                 (city_data_df["Cloudiness"] >= 0 )&
                                                 (city_data_df["Cloudiness"] < 40 )
                                                 ,:]


# Drop any rows with null values
ideal_weather_df=ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,nouadhibou,20.9310,-17.0347,22.99,53,3,5.66,MR,1678137685
58,58,bhachau,23.2833,70.3500,25.37,37,33,2.64,IN,1678137699
108,108,pisco,-13.7000,-76.2167,28.03,57,0,8.23,PE,1678137710
119,119,lazaro cardenas,17.9583,-102.2000,27.94,56,27,5.06,MX,1678137713
121,121,cabo san lucas,22.8909,-109.9124,26.45,38,0,3.60,MX,1678137486
201,201,salalah,17.0151,54.0924,25.05,33,0,1.03,OM,1678137738
210,210,vallenar,-28.5708,-70.7581,27.72,37,0,3.55,CL,1678137741
220,220,najran,17.4924,44.1277,20.36,30,0,1.32,SA,1678137746
229,229,springdale,36.1867,-94.1288,24.17,47,0,4.63,US,1678137689
285,285,maldonado,-34.6667,-54.9167,27.66,37,15,7.46,UY,1678137767


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City','Country','Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,nouadhibou,MR,20.9310,-17.0347,53,
58,bhachau,IN,23.2833,70.3500,37,
108,pisco,PE,-13.7000,-76.2167,57,
119,lazaro cardenas,MX,17.9583,-102.2000,56,
121,cabo san lucas,MX,22.8909,-109.9124,38,
201,salalah,OM,17.0151,54.0924,33,
210,vallenar,CL,-28.5708,-70.7581,37,
220,najran,SA,17.4924,44.1277,30,
229,springdale,US,36.1867,-94.1288,47,
285,maldonado,UY,-34.6667,-54.9167,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" :"accommodation.hotel",
    "apiKey" : geoapify_key,
    "limit" : 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nouadhibou - nearest hotel: Hotel valencia
bhachau - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
cabo san lucas - nearest hotel: Comfort Rooms
salalah - nearest hotel: Muscat International Hotel
vallenar - nearest hotel: Hotel del Marqués
najran - nearest hotel: No hotel found
springdale - nearest hotel: Super 8 by Wyndham Springdale AR
maldonado - nearest hotel: No hotel found
chai badan - nearest hotel: Narai Grand Hotel
sherman - nearest hotel: Travelers Inn
avera - nearest hotel: No hotel found
bol - nearest hotel: No hotel found
libenge - nearest hotel: Hôtel Wilo et Frères
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
santa cruz del sur - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
porbandar - nearest hotel: Toran Tourist Bungalow
minab - nearest hotel: هتل صدف
hamilton - nearest hotel: North Vista Manor
valdosta - nearest hotel: Comfor

,City,Country,Lat,Lng,Humidity,Hotel Name
18,nouadhibou,MR,20.9310,-17.0347,53,Hotel valencia
58,bhachau,IN,23.2833,70.3500,37,No hotel found
108,pisco,PE,-13.7000,-76.2167,57,La Portada
119,lazaro cardenas,MX,17.9583,-102.2000,56,Hotel Sol del Pacífico
121,cabo san lucas,MX,22.8909,-109.9124,38,Comfort Rooms
201,salalah,OM,17.0151,54.0924,33,Muscat International Hotel
210,vallenar,CL,-28.5708,-70.7581,37,Hotel del Marqués
220,najran,SA,17.4924,44.1277,30,No hotel found
229,springdale,US,36.1867,-94.1288,47,Super 8 by Wyndham Springdale AR
285,maldonado,UY,-34.6667,-54.9167,37,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
                "Lng",
                "Lat",
                 geo = True,
                 tiles = "OSM",
                 frame_width = 500,
                 frame_height = 600,
                 size = "Humidity",            
                 color = "City",
                 hover_cols = ["Hotel Name" , "Country"]
                    ) 

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)